# Parse Evos

This is the same as the parse_evos.py script, but maintains load memory, so you can debug script configs without having to reload the data every time.

In [ ]:
%reload_ext autoreload
%autoreload 2

import logging.config
from pathlib import Path

from omegaconf import OmegaConf
import matplotlib.pyplot as plt

from cambrian.utils.logger import get_logger

from parse_evos import ParseEvosConfig, try_load_pickle, load_data, save_data, run_plot, update_plots

# plt.ioff()  # Turn off interactive mode
%matplotlib agg

In [ ]:
config_name = "tools/parse_evos/env_vs_hw2"
base_dir = Path.cwd().parent.parent.absolute()

In [ ]:
folder = input("Enter the folder name: ")
assert (base_dir / folder).exists(), f"Folder {folder} does not exist in {base_dir}."

In [ ]:
overrides = [f"folder={base_dir / folder}", "force=True", "debug=False"]
config, hydra_config = ParseEvosConfig.compose(base_dir / "configs", config_name, overrides=overrides, return_hydra_config=True)

logging_config = OmegaConf.to_container(hydra_config.job_logging)
del logging_config["handlers"]["console"]
del logging_config["handlers"]["file"]
logging.config.dictConfig(logging_config)
assert not (config.debug and config.quiet), "Cannot be both debug and quiet."
if config.debug:
    get_logger().setLevel("DEBUG")
elif config.quiet:
    get_logger().setLevel("WARNING")

get_logger().info(f"Loaded folder: {folder}")

In [ ]:
if config.force or (data := try_load_pickle(config.output, "data.pkl")) is None:
    data = load_data(config)

    if not config.no_save:
        save_data(data, config.output, "data.pkl")
assert data is not None

In [ ]:
%%capture

# Save
run_plot(config, data)
update_plots(config, save=False, show=True)

In [ ]:
# Save
run_plot(config, data)
update_plots(config, save=True, show=False)